In [30]:
import numpy as np
import pandas as pd
import re
from collections import defaultdict
from nltk.stem import PorterStemmer
import nltk
import math


In [31]:
# data = pd.read_csv('combined_relevant_dataset.csv')
data = pd.read_csv('test_dataset_labeled.csv')
data['Input Data'] = data['Name'].apply(str) + " " + data['Genres'].apply(str) + " " + data['Type'].apply(str) + " " + data['Aired'].apply(str) + " " + data['Studios'].apply(str) + " " + data['sypnopsis'].apply(str)

In [32]:
# input=data['Input Data'].tolist()
# len(input)
import spacy
import string

spacy_nlp = spacy.load('en_core_web_sm')

punctuations = string.punctuation
stop_words = spacy.lang.en.stop_words.STOP_WORDS

#TODO add stemming
def tokenizer(sentence):
    sentence = re.sub('\'','',sentence)
    sentence = re.sub('\w*\d\w*','',sentence)
    sentence = re.sub(' +',' ',sentence)
    sentence = re.sub(r'\n: \'\'.*','',sentence)
    sentence = re.sub(r'\n!.*','',sentence)
    sentence = re.sub(r'^:\'\'.*','',sentence)
    sentence = re.sub(r'\n',' ',sentence)
    sentence = re.sub(r'[^\w\s]',' ',sentence)
    tokens = spacy_nlp(sentence)
    tokens = [word.lemma_.lower().strip() if word.lemma_ != "-PRON-" else word.lower_ for word in tokens]
    tokens = [word for word in tokens if word not in stop_words and word not in punctuations and len(word) > 2]
    # ps = PorterStemmer()
    # tokens = [ps.stem(word) for word in tokens]
    return tokens
k1 = 1.5
b = 0.75
data['input_data_tokenized'] = data['Input Data'].map(tokenizer)
# collection = {data.loc[i, ["MAL_ID"]].values[0]: tokenizer(data.loc[i, ["Input Data"]].values[0]) for i in data.index}

In [33]:
corpus = {data.loc[i, ["MAL_ID"]].values[0]: data.loc[i, ["input_data_tokenized"]].values[0]for i in data.index}

In [69]:
def bm25_init(corpus, eps=0.25):
  corpus_size = 0
  avgdl = 0
  doc_freqs = {}
  idf = {}
  doc_len = {}
  nd = {}  # word -> number of documents with word
  num_doc = 0
  for key, document in corpus.items():
    corpus_size += 1
    doc_len[key] = len(document)
    num_doc += len(document)
    frequencies = {}
    for word in document:
      if word not in frequencies:
        frequencies[word] = 0
      frequencies[word] += 1
    doc_freqs[key] = frequencies
    for word, freq in frequencies.items():
      if word not in nd:
        nd[word] = 0
      nd[word] += 1
  avgdl = float(num_doc) / corpus_size
  # collect idf sum to calculate an average idf for epsilon value
  idf_sum = 0
  # collect words with negative idf to set them a special epsilon value.
  # idf can be negative if word is contained in more than half of documents
  negative_idfs = []
  for word, freq in nd.items():
    temp_idf = math.log(corpus_size - freq + 0.5) - math.log(freq + 0.5)
    idf[word] = temp_idf
    idf_sum += temp_idf
    if temp_idf < 0:
      negative_idfs.append(word)
  average_idf = float(idf_sum) / len(idf)
  eps = eps * average_idf
  for word in negative_idfs:
    idf[word] = eps
  
  return corpus_size, avgdl, doc_freqs, idf, doc_len


def get_score(query, index, doc_freqs, doc_len, idf, avgdl, k1, b):
  score = 0.0
  df_dict = doc_freqs[index]
  numerator_constant = k1 + 1
  denominator_constant = k1 * (1 - b + b * doc_len[index] / avgdl)
  for word in query:
    if word in df_dict:
      df = doc_freqs[index][word]
      temp_idf = idf[word]
      score += (temp_idf * df * numerator_constant) / (df + denominator_constant)
  return score


def get_scores(query, corpus, doc_freqs, doc_len, idf, avgdl, k1, b):
  query = tokenizer(query)
  scores = {index: get_score(query, index, doc_freqs, doc_len, idf, avgdl, k1, b) for index in corpus}
  return scores


def bm25_search(query, corpus, doc_freqs, doc_len, idf, avgdl, k1=1.5, b=0.75, num_best=5):
  scores = get_scores(query, corpus, doc_freqs, doc_len, idf, avgdl, k1, b)
  results = sorted(scores.items(), key=lambda item: item[1], reverse=True)[:num_best]
  return results

In [71]:
corpus_size, avgdl, doc_freqs, idf, doc_len = bm25_init(corpus)
results = bm25_search("bakugan shounen brawlers", corpus, doc_freqs, doc_len, idf, avgdl)

[(41405, 5.64351769518006), (42519, 1.8280201262445748), (1253, 1.786124022100845), (12857, 1.7461053106594162), (915, 1.6534879234276645)]


In [5]:
# clean and uniform the text
def preprocess_string(s):
  s = re.sub(r"won't", "will not", s)
  s = re.sub(r"can\'t", "can not", s)
  s = re.sub(r"n\’t", " not", s)
  s = re.sub(r"\’re", " are", s)
  s = re.sub(r"\’s", " is", s)
  s = re.sub(r"\’d", " would", s)
  s = re.sub(r"\’ll", " will", s)
  s = re.sub(r"\’t", " not", s)
  s = re.sub(r"\’ve", " have", s)
  s = re.sub(r"\’m", " am", s)
  s = re.sub(r'[0–9]+', '', s)
  s=re.sub(r'[^\w\s]','' '', s)
  s=stem.stem(s)
  return s

In [61]:
# def preprocess(docs):
#   new_docs=[]
#   for doc in docs:
#     new_doc=[]
#     tmp={}
#     for word in doc.lower().split():
#       if word not in stop_words:
#         a=preprocess_string(word)
#         a=stem.stem(a)
#         tmp[a]=tmp.get(a,0)+1
#         new_doc.append(a)
#     f.append(tmp)
#     new_docs.append(new_doc)
#   return new_docs


In [13]:
f=[]
tf={}
idf={}
k1=1.5
b=0.75

def init(docs):
  D=len(docs)
  for doc in docs:
    tmp={}
    for word in doc.lower().split():
      if word not in stop_words:
        a=preprocess_string(word)
        a=stem.stem(a)
        tmp[a]=tmp.get(a,0)+1
    f.append(tmp)
    for k in tmp.keys():
      tf[k]=tf.get(k,0)+1
  for k,v in tf.items():
    idf[k]=math.log(D-v+0.5) - math.log(v+0.5)

  avgd=sum(len(doc)+0.0 for doc in docs)/D
  return D,avgd

In [18]:
def sim(doc,index):
  score =0.0
  for word in doc:
    if word not in f[index]:
      continue
    else:
      d=len(f[index])
      score +=(idf[word]* f[index][word]*(k1+1)/(f[index][word]+k1*(1-b+b*d/avgd)))
  return score
def sim_overall(doc):
  scores=[]
  for index in range(D):
    score =sim(doc,index)
    scores.append(score)
  return scores

In [14]:
D,avgd=init(input)


In [15]:
D,avgd

(16214, 428.74935241149626)

In [ ]:
tf

In [ ]:
idf

In [38]:
##
import heapq
res=sim_overall('comedy')
re1=map(res.index,heapq.nlargest(5,res))
re2=heapq.nlargest(5,res)
print(list(re1),re2)

[4263, 11929, 5969, 5528, 5529] [23.399245271890756, 20.37090799954332, 18.879770132141424, 15.514535571478126, 15.457183402492502]


In [41]:
# re1 is the index of the dataframe row number for top 5 highest, re2 is the score
def BM25_search(string):
  res=sim_overall(string)
  re1=map(res.index,heapq.nlargest(5,res))
  re2=heapq.nlargest(5,res)
  return(list(re1),re2)

In [42]:
BM25_search('school')

[4263, 903, 348, 15435, 521] [40.71399743914366, 26.941557483397894, 26.611868662928092, 26.199655764695105, 24.512150626047088]


In [43]:
data.loc[4263]

MAL_ID                                                     7245
Name                  Sekai Meisaku Douwa: Wow! Maerchen Oukoku
Score                                                      6.52
Genres                                                  Fantasy
sypnopsis     Based on Western tales from the usual suspects...
Input Data    Sekai Meisaku Douwa: Wow! Maerchen Oukoku Fant...
Name: 4263, dtype: object